In [1]:
import pandas as pd

In [2]:
#Configuración
RUTA_BASE_INPUT = './INPUT/'
NOMBRE_CONSULTA_SA = 'CONSULTA_SQL.csv'
NOMBRE_CONSOLIDADO = 'CONSOLIDADO_DATOS_IMPORTANTES.xlsx'
# Mapa
COLUMNAS = {
    'LLAVE_MINTIC': 'MINTIC',
    'CODIGO_UIS': 'ESTUDIANTE',
    'NUMERO_DE_DOCUMENTO': 'DOCUMENTO',
    'APELLIDOS': 'NOMBRES',
    'NOMBRES': 'APELLIDOS',
    'EMAIL': 'EMAIL',
    'TELEFONO_MINTIC': 'TELEFONO',
    'ASIGNATURA': 'ASIGNATURA 1',
    'GRUPO': 'GRUPO 1',
    'DOCENTE_PYTHON': 'DOCENTE',
    'HORA_INICIAL': 'HORA INICIAL',
    'DURACION': 'DURACION',
    # INGLES
    'ASIGNATURA2': 'ASIGNATURA 2',
    'GRUPO2': 'GRUPO 2',
    'DOCENTE_INGLES': 'DOCENTE.1',
    'HORA_INICIAL2': 'HORA INICIAL.1',
    'DURACION2': 'DURACION.1',
    # COACH
    'ASIGNATURA3': 'ASIGNATURA 3',
    'GRUPO3': 'GRUPO 3',
    'DOCENTE_COACH': 'DOCENTE.2',
    'HORA_INICIAL3': 'HORA INICIAL.2',
    'DIA': 'DIA',
    'DURACION3': 'DURACION.2'
}
COLUMNAS_ADICIONALES = {
    'pssBiblio': 'PSSW_BIBLIO',
    'hIniCad': 'HORA_INICIAL_',
    'hFin': 'HORA_FINAL',
    'hFinCad': 'HORA_FINAL_',
    'hIniCad2': 'HORA_INICIAL2_',
    'hFin2': 'HORA_FINAL2',
    'hFinCad2': 'HORA_FINAL2_',
    'hIniCad3': 'HORA_INICIAL3_',
    'hFin3': 'HORA_FINAL3',
    'hFinCad3': 'HORA_FINAL3_',
    'telForm': 'TELEFONO_FORMULARIO',
    'telSis': 'TELEFONO_SISTEMA_ACADEMICO'
}
ORDEN = [
    'CODIGO_UIS',
    'NUMERO_DE_DOCUMENTO',
    'PSSW_BIBLIO',
    'NOMBRES',
    'APELLIDOS',
    'EMAIL',
    'ASIGNATURA',
    'GRUPO',
    'HORA_INICIAL',
    'HORA_INICIAL_',
    'HORA_FINAL',
    'HORA_FINAL_',
    'DURACION',
    'ASIGNATURA2',
    'GRUPO2',
    'HORA_INICIAL2',
    'HORA_INICIAL2_',
    'HORA_FINAL2',
    'HORA_FINAL2_',
    'DURACION2',
    'ASIGNATURA3',
    'GRUPO3',
    'DIA',
    'HORA_INICIAL3',
    'HORA_INICIAL3_',
    'HORA_FINAL3',
    'HORA_FINAL3_',
    'DURACION3',
    'LLAVE_MINTIC',
    'TELEFONO_MINTIC',
    'TELEFONO_FORMULARIO',
    'TELEFONO_SISTEMA_ACADEMICO',
    'DOCENTE_PYTHON',
    'DOCENTE_INGLES',
    'DOCENTE_COACH'
]
DIAS = [
    'Lunes',
    'Martes',
    'Miércoles',
    'Jueves',
    'Viernes'
]


In [3]:
# Lectura
consulta = pd.read_csv(RUTA_BASE_INPUT + NOMBRE_CONSULTA_SA, sep=';', encoding='latin-1')
consolidado = pd.read_excel(RUTA_BASE_INPUT + NOMBRE_CONSOLIDADO, engine='openpyxl')

In [4]:
# Funciones
def buscarEnDf(df, columnaKey, key, columna):
    query = df[df[columnaKey] == key][columna]
    if len(query) > 0:        
        return query.iloc[0]
    return -1
def genBibPass(documento):
    documento = str(documento)
    return documento[len(documento) - 4:]

In [5]:
# Procesamiento
listas = {}
for columna in COLUMNAS_ADICIONALES:
    listas[columna] = []
consultaE = consulta.copy()
for index, row in consultaE.iterrows():
    listas['pssBiblio'].append(genBibPass(row[COLUMNAS['NUMERO_DE_DOCUMENTO']]))
    listas['hIniCad'].append(str(row[COLUMNAS['HORA_INICIAL']]) + ':00:00')
    listas['hFin'].append(str(row[COLUMNAS['HORA_INICIAL']] + row[COLUMNAS['DURACION']]))
    listas['hFinCad'].append(str(row[COLUMNAS['HORA_INICIAL']] + row[COLUMNAS['DURACION']]) + ':00:00')
    listas['hIniCad2'].append(str(row[COLUMNAS['HORA_INICIAL2']]) + ':00:00')
    listas['hFinCad2'].append(str(row[COLUMNAS['HORA_INICIAL2']] + row[COLUMNAS['DURACION2']]) + ':00:00')
    listas['hFin2'].append(str(row[COLUMNAS['HORA_INICIAL2']] + row[COLUMNAS['DURACION2']]))
    listas['hIniCad3'].append(str(row[COLUMNAS['HORA_INICIAL3']]) + ':00:00')
    listas['hFin3'].append(str(row[COLUMNAS['HORA_INICIAL3']] + row[COLUMNAS['DURACION3']]))
    listas['hFinCad3'].append(str(row[COLUMNAS['HORA_INICIAL3']] + row[COLUMNAS['DURACION3']]) + ':00:00')
    consultaE.at[index, COLUMNAS['DIA']] = DIAS[row[COLUMNAS['DIA']] - 1]
    telForm = buscarEnDf(consolidado, 'CODIGO', row[COLUMNAS['CODIGO_UIS']], 'TELEFONO')
    if telForm != -1:        
        listas['telForm'].append(telForm)
    else:
        listas['telForm'].append('')
    listas['telSis'].append('dummy')
for columnaNueva, lista in listas.items():
    consultaE[columnaNueva] = pd.Series(lista)

In [6]:
# Renombre
columnasAuxiliares = {v: k for k, v in COLUMNAS.items()}
columnasRenombradas = {**columnasAuxiliares, **COLUMNAS_ADICIONALES}
consultaERen = consultaE.rename(columns = columnasRenombradas)

In [7]:
consultaERenOrdenada = consultaERen[ORDEN]

In [8]:
from datetime import date
consultaERenOrdenada.to_excel('./OUTPUT/HorariosEstudiantes_VX_' + str(date.today()) + '.xlsx', index = False)